In [1]:
import os
import tensorflow as tf
from tensorflow import keras as keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


Data Preparing

In [2]:
data_dir_path = "../data"

data = pd.read_csv(f"{data_dir_path}/emnist-byclass-mapping.txt", sep=" ", header=None)
train_df = pd.read_csv(f'{data_dir_path}/emnist-byclass-train.csv')
test_df = pd.read_csv(f"{data_dir_path}/emnist-byclass-test.csv")

data.columns = ['Target','ASCII']
test_df.rename(columns={'18':'Target'}, inplace=True)
train_df.rename(columns={'35':'Target'}, inplace=True)

col=[]
for i in data.ASCII.tolist():
    col.append(chr(i))

data['ASCII_Target'] = pd.Series(col)

In [3]:
y_train = np.array(train_df.pop('Target'))
X_train = train_df
y_test = np.array(test_df.pop('Target'))
X_test = test_df

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train.values.reshape([-1, 28, 28, 1])
X_test = X_test.values.reshape([-1, 28, 28, 1])

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [4]:
print(X_train.shape, X_val.shape, X_test.shape)

(558344, 28, 28, 1) (139587, 28, 28, 1) (116322, 28, 28, 1)


In [5]:
# # expand new axis, channel axis 
# X_train = np.expand_dims(X_train, axis=-1)
# X_val = np.expand_dims(X_val, axis=-1)
# X_test = np.expand_dims(X_test, axis=-1)

# # [optional]: we may need 3 channel (instead of 1)
# X_train = np.repeat(X_train, 3, axis=-1)
# X_val = np.repeat(X_val, 3, axis=-1)
# X_test = np.repeat(X_test, 3, axis=-1)

# # it's always better to normalize 
# X_train = X_train.astype('float32') / 255
# X_val = X_val.astype('float32') / 255
# X_test = X_test.astype('float32') / 255

# # resize the input shape , i.e. old shape: 28, new shape: 32
# X_train = tf.image.resize(X_train, [32,32]) # if we want to resize 
# X_val = tf.image.resize(X_val, [32,32]) # if we want to resize 
# X_test = tf.image.resize(X_test, [32,32]) # if we want to resize 

# # one hot 
# y_train = tf.keras.utils.to_categorical(y_train , num_classes=10)
# y_val = tf.keras.utils.to_categorical(y_val , num_classes=10)
# y_test = tf.keras.utils.to_categorical(y_test , num_classes=10)

# print(X_train.shape, y_train.shape)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [5]:
class ResnetBlock(keras.models.Model):
    """
    A standard resnet block.
    """

    def __init__(self, channels: int, down_sample=False):
        """
        channels: same as number of convolution kernels
        """
        super().__init__()

        self.__channels = channels
        self.__down_sample = down_sample
        self.__strides = [2, 1] if down_sample else [1, 1]

        KERNEL_SIZE = (3, 3)
        # use He initialization, instead of Xavier (a.k.a 'glorot_uniform' in Keras), as suggested in [2]
        INIT_SCHEME = "he_normal"

        self.conv_1 = keras.layers.Conv2D(self.__channels, strides=self.__strides[0],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_1 = keras.layers.BatchNormalization()
        self.conv_2 = keras.layers.Conv2D(self.__channels, strides=self.__strides[1],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_2 = keras.layers.BatchNormalization()
        self.merge = keras.layers.Add()

        if self.__down_sample:
            # perform down sampling using stride of 2, according to [1].
            self.res_conv = keras.layers.Conv2D(
                self.__channels, strides=2, kernel_size=(1, 1), kernel_initializer=INIT_SCHEME, padding="same")
            self.res_bn = keras.layers.BatchNormalization()

    def call(self, inputs):
        res = inputs

        x = self.conv_1(inputs)
        x = self.bn_1(x)
        x = tf.nn.relu(x)
        x = self.conv_2(x)
        x = self.bn_2(x)

        if self.__down_sample:
            res = self.res_conv(res)
            res = self.res_bn(res)

        # if not perform down sample, then add a shortcut directly
        x = self.merge([x, res])
        out = tf.nn.relu(x)
        return out


class ResNet18(keras.models.Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        self.conv_1 = keras.layers.Conv2D(64, (7, 7), strides=2,
                             padding="same", kernel_initializer="he_normal")
        self.init_bn = keras.layers.BatchNormalization()
        self.pool_2 = keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding="same")
        self.res_1_1 = ResnetBlock(64)
        self.res_1_2 = ResnetBlock(64)
        self.res_2_1 = ResnetBlock(128, down_sample=True)
        self.res_2_2 = ResnetBlock(128)
        self.res_3_1 = ResnetBlock(256, down_sample=True)
        self.res_3_2 = ResnetBlock(256)
        self.res_4_1 = ResnetBlock(512, down_sample=True)
        self.res_4_2 = ResnetBlock(512)
        self.avg_pool = keras.layers.GlobalAveragePooling2D()
        self.flat = keras.layers.Flatten()
        self.fc = keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2, self.res_4_1, self.res_4_2]:
            out = res_block(out)
        out = self.avg_pool(out)
        out = self.flat(out)
        out = self.fc(out)
        return out


resnet_model = ResNet18(62)
resnet_model.build(input_shape = (None,28,28,1))
#use categorical_crossentropy since the label is one-hot encoded

# from keras.optimizers import SGD
# opt = SGD(learning_rate=0.1,momentum=0.9,decay = 1e-04) #parameters suggested by He [1]
resnet_model.compile(optimizer = "adam",loss='categorical_crossentropy', metrics=["accuracy"]) 
resnet_model.summary()

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "res_net18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  3200      
                                                                 
 batch_normalization (BatchN  multiple                 256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 resnet_block (ResnetBlock)  multiple          

In [6]:
# input = tf.keras.Input(shape=(32,32,3))
# efnet = tf.keras.applications.ResNet50(weights='imagenet',
#                                              include_top = False, 
#                                              input_tensor = input)
# # Now that we apply global max pooling.

# # Finally, we add a classification layer.
# number_output = tf.keras.layers.Dense(10, activation='softmax', use_bias=True)()
# upper_output = tf.keras.layers.Dense(26,  activation='softmax', use_bias=True)()
# lower_output = tf.keras.layers.Dense(26,  activation='softmax', use_bias=True)()

# # bind all
# func_model = tf.keras.Model(efnet.input, output)

In [12]:
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

es = EarlyStopping(patience= 8, restore_best_weights=True, monitor="val_acc")
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("new_model.h5", save_best_only=True)

batch_size = 256
#I did not use cross validation, so the validate performance is not accurate.
STEPS = len(X_train) / 256

history = resnet_model.fit(aug.flow(X_train,y_train,batch_size = batch_size), 
      steps_per_epoch=STEPS, 
      batch_size = batch_size, 
      epochs=50, 
      validation_data=(X_train, y_train),
      callbacks=[es])

Epoch 1/50
   4/2181 [..............................] - ETA: 2:27:02 - loss: 4.7207 - accuracy: 0.0947

KeyboardInterrupt: 